In [1]:
# Reset autoreload completely
try:
    %autoreload 0  # Disable autoreload
    %reload_ext autoreload
    %autoreload 2  # Re-enable with full reload
except:
    %load_ext autoreload
    %autoreload 2

from nirs4all.pipeline.runner import PipelineRunner
from nirs4all.pipeline.config import PipelineConfig
from nirs4all.dataset import dataset
# from sample import dataset_config, pipeline_config
from sample_classif import dataset_config, pipeline_config
from nirs4all.dataset.loader import get_dataset
from nirs4all.controllers.registry import reset_registry
from nirs4all.controllers import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler

data = get_dataset(dataset_config)
config = PipelineConfig(pipeline_config)
runner = PipelineRunner()
res_dataset, history, pipeline = runner.run(config, data)

print("✅ Simple transformers pipeline completed successfully!")
print(f"Final dataset: {res_dataset}")


Loading dataset:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
📊 Dataset: classification
Features (samples=66, sources=1):
- Source 0: (66, 1, 2152), processings=['raw'], min=-0.032, max=0.948, mean=0.34, var=0.106)
Targets: (samples=66, targets=1, processings=['numeric'])
- numeric: min=0.0, max=9.0, mean=2.894
Indexes:
- "train", ['raw']: 48 samples
- "test", ['raw']: 18 samples
🚀 Starting pipeline config_d4e86654 on dataset classification
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
🔄 Running 11 steps in sequential mode
🔷 Step 1: MinMaxScaler(feature_range=[0.1, 0.8])
🔹 Executing contro

d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


🔍 Optimizing 3 parameters with grid search (4 trials)...
(108, 6456) (108,) (12, 6456) (12, 1)


d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


(36, 6456) (36,) (12, 6456) (12, 1)
(36, 6456) (36,) (12, 6456) (12, 1)
(36, 6456) (36,) (12, 6456) (12, 1)
💾 Saved 5_finetuned_RandomForestClassifier_6.pkl to results\classification\config_d4e86654\5_finetuned_RandomForestClassifier_6.pkl
💾 Saved 5_predictions_finetuned_7.csv to results\classification\config_d4e86654\5_predictions_finetuned_7.csv
💾 Saved 5_trained_RandomForestClassifier_9_simple_cv_fold1.pkl to results\classification\config_d4e86654\5_trained_RandomForestClassifier_9_simple_cv_fold1.pkl
💾 Saved 5_predictions_trained_10_simple_cv_fold1.csv to results\classification\config_d4e86654\5_predictions_trained_10_simple_cv_fold1.csv
💾 Saved 5_trained_RandomForestClassifier_12_simple_cv_fold2.pkl to results\classification\config_d4e86654\5_trained_RandomForestClassifier_12_simple_cv_fold2.pkl
💾 Saved 5_predictions_trained_13_simple_cv_fold2.csv to results\classification\config_d4e86654\5_predictions_trained_13_simple_cv_fold2.csv
💾 Saved 5_trained_RandomForestClassifier_15_simp

d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\covariance\_shrunk_covariance.py:349: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\covariance\_empirical_covariance.py:102: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\covariance\_shrunk_covariance.py:349: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\sklearn\covariance\_empirical_covariance.py:102: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
d

🏆 Training with best parameters: {'solver': 'eigen', 'shrinkage': 'auto'}
(36, 6456) (36,) (12, 6456) (12, 1)
(36, 6456) (36,) (12, 6456) (12, 1)
(36, 6456) (36,) (12, 6456) (12, 1)
💾 Saved 6_finetuned_LinearDiscriminantAnalysis_18.pkl to results\classification\config_d4e86654\6_finetuned_LinearDiscriminantAnalysis_18.pkl
💾 Saved 6_predictions_finetuned_19.csv to results\classification\config_d4e86654\6_predictions_finetuned_19.csv
💾 Saved 6_trained_LinearDiscriminantAnalysis_21_simple_cv_fold1.pkl to results\classification\config_d4e86654\6_trained_LinearDiscriminantAnalysis_21_simple_cv_fold1.pkl
💾 Saved 6_predictions_trained_22_simple_cv_fold1.csv to results\classification\config_d4e86654\6_predictions_trained_22_simple_cv_fold1.csv
💾 Saved 6_trained_LinearDiscriminantAnalysis_24_simple_cv_fold2.pkl to results\classification\config_d4e86654\6_trained_LinearDiscriminantAnalysis_24_simple_cv_fold2.pkl
💾 Saved 6_predictions_trained_25_simple_cv_fold2.csv to results\classification\confi

Traceback (most recent call last):
  File "D:\Workspace\ML\NIRS\nirs4all\nirs4all\controllers\tensorflow\op_model.py", line 135, in _create_model_from_function
    model = model_function(input_shape, params)
  File "D:\Workspace\ML\NIRS\nirs4all\nirs4all\operators\models\cirad_tf.py", line 681, in nicon_classification
    model.add(Dense(units=num_classes, activation="softmax"))
  File "d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\keras\src\models\sequential.py", line 122, in add
    self._maybe_rebuild()
  File "d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\keras\src\models\sequential.py", line 149, in _maybe_rebuild
    self.build(input_shape)
  File "d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\keras\src\layers\layer.py", line 230, in build_wrapper
    original_build_method(*args, **kwargs)
  File "d:\Workspace\ML\NIRS\nirs4all\.venv\lib\site-packages\keras\src\models\sequential.py", line 195, in build
    x = layer(x)
  File "d:\Workspace\ML\NIRS\nirs4all\

❌ Pipeline config_d4e86654 on dataset classification failed: 
Pipeline step failed: Error creating model from function nicon_classification: Invalid dtype: TrackedDict


RuntimeError: Pipeline step failed: Error creating model from function nicon_classification: Invalid dtype: TrackedDict

In [11]:
# Reset autoreload completely
try:
    %autoreload 0  # Disable autoreload
    %reload_ext autoreload
    %autoreload 2  # Re-enable with full reload
except:
    %load_ext autoreload
    %autoreload 2

from nirs4all.pipeline.runner import PipelineRunner
from nirs4all.pipeline.config import PipelineConfig
from nirs4all.dataset import dataset
from sample import dataset_config, pipeline_config, generator_config
# from sample_classif import dataset_config, pipeline_config
from nirs4all.dataset.loader import get_dataset
from nirs4all.controllers.registry import reset_registry
from nirs4all.controllers import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler

data = get_dataset(dataset_config)
config = PipelineConfig(generator_config)
if config.has_configurations:
    print(f"Generated {len(config.steps)} configurations.")
    # for i, step in enumerate(config.steps):
        # print(config)
# runner = PipelineRunner()
# res_dataset, history, pipeline = runner.run(config, data)

# print("✅ Simple transformers pipeline completed successfully!")
# print(f"Final dataset: {res_dataset}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading dataset:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
⚠️ Dataset does not have data for train_group.
⚠️ Dataset does not have data for test_group.
📊 Dataset: regression
Features (samples=189, sources=1):
- Source 0: (189, 1, 2151), processings=['raw'], min=-0.265, max=1.436, mean=0.466, var=0.149)
Targets: (samples=189, targets=1, processings=['numeric'])
- numeric: min=1.33, max=128.31, mean=30.779
Indexes:
- "train", ['raw']: 130 samples
- "test", ['raw']: 59 samples
Configuration generates 80 configurations.
Generated 80 configurations.


In [ ]:
data = get_dataset(dataset_config)
config = PipelineConfig(pipeline_config)
runner = PipelineRunner()

pipeline_path = "results/sample_data/config_89f2f434"
pred_dataset, pred_context = PipelineRunner.predict(
    path=pipeline_path,
    dataset=data,  # Use same data for testing
    verbose=1
)
print("✅ Prediction mode completed successfully!")

predictions_obj = pred_dataset._predictions
pred_count = len(predictions_obj._predictions)
print(f"🎯 Generated {pred_count} prediction records")


# 📊 Prediction Visualization Testing

Test the new `PredictionVisualizer` class for analyzing and displaying prediction results graphically.

In [ ]:
# Import and test the PredictionVisualizer
from nirs4all.dataset.prediction_visualizer import PredictionVisualizer
import matplotlib.pyplot as plt

print("🔍 Testing PredictionVisualizer with current prediction results...")
viz = PredictionVisualizer(predictions_obj)
print("\n" + viz.summary_report())

# Show only best predictions
viz.plot_filtered_predictions(prediction_filter='best_only', metric='r2')

# Performance matrix with only global predictions
viz.plot_performance_matrix(prediction_filter='global_only')

# Bar chart comparing all prediction types
viz.plot_filtered_predictions(chart_type='bar', metric='rmse')

In [ ]:
# Test 1: Performance Matrix Visualization
print("📈 Testing Performance Matrix...")

try:
    if 'viz' in globals():
        # Create performance matrix plot

        plt.show()
        print("✅ Performance matrix plot created successfully!")

    else:
        print("❌ Visualizer not initialized. Run previous cell first.")

except Exception as e:
    print(f"❌ Error creating performance matrix: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test 2: Multi-Metric Comparison
print("📊 Testing Multi-Metric Comparison...")

try:
    if 'viz' in globals():
        # Create multi-metric comparison plot
        fig2 = viz.plot_multi_metric_comparison(
            metrics=['rmse', 'mae', 'r2', 'mse'],  # Show all metrics
            sort_by='r2',      # Sort by R² (best first)
            ascending=False,   # Higher R² is better
            figsize=(16, 10)
        )

        plt.show()
        print("✅ Multi-metric comparison plot created successfully!")

    else:
        print("❌ Visualizer not initialized. Run previous cell first.")

except Exception as e:
    print(f"❌ Error creating multi-metric plot: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test 3: Prediction Scatter Plots
print("🎯 Testing Prediction Scatter Plots...")

try:
    if 'viz' in globals():
        # Create scatter plot of true vs predicted values
        fig3 = viz.plot_prediction_scatter(
            figsize=(14, 8)  # Show scatter plots for all model-config combinations
        )

        plt.show()
        print("✅ Prediction scatter plots created successfully!")

    else:
        print("❌ Visualizer not initialized. Run previous cell first.")

except Exception as e:
    print(f"❌ Error creating scatter plots: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test 4: Demonstrate different sorting and filtering options
print("⚙️ Testing Advanced Visualization Options...")

try:
    if 'viz' in globals():
        # Test different sorting options
        print("\n🔄 Creating performance matrix sorted by best config:")
        fig4a = viz.plot_performance_matrix(
            metric='r2',         # Use R² metric
            sort_by='config',    # Sort by config performance
            ascending=False,     # Best (highest R²) first
            figsize=(10, 6),
            cmap='RdYlGn',      # Green is better for R²
            show_values=True
        )
        plt.show()

        print("\n📈 Creating RMSE matrix sorted by model:")
        fig4b = viz.plot_performance_matrix(
            metric='rmse',       # Use RMSE metric
            sort_by='model',     # Sort by model performance
            ascending=True,      # Best (lowest RMSE) first
            figsize=(10, 6),
            cmap='RdYlGn_r',    # Red is worse for RMSE
            show_values=True
        )
        plt.show()

        print("✅ Advanced visualization options tested successfully!")

    else:
        print("❌ Visualizer not initialized. Run previous cell first.")

except Exception as e:
    print(f"❌ Error in advanced visualization: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Debug: Explore dictionary structure
print("🔍 Exploring prediction dictionary structure...")

try:
    if 'pred_dataset' in globals() and hasattr(pred_dataset, '_predictions'):
        predictions_obj = pred_dataset._predictions
        pred_data = predictions_obj._predictions

        print(f"Dictionary keys: {list(pred_data.keys())}")

        # Explore each key
        for key, value in pred_data.items():
            print(f"\nKey: {key}")
            print(f"  Type: {type(value)}")
            if hasattr(value, '__len__'):
                print(f"  Length: {len(value)}")

            # Show first few items or structure
            if isinstance(value, (list, tuple)) and len(value) > 0:
                print(f"  First item type: {type(value[0])}")
                print(f"  First item: {value[0]}")
            elif isinstance(value, dict):
                print(f"  Dict keys: {list(value.keys())}")
                for subkey, subvalue in list(value.items())[:2]:  # Show first 2 items
                    print(f"    {subkey}: {type(subvalue)} = {subvalue}")
            else:
                print(f"  Value: {value}")

        # Try the get_predictions method
        print(f"\n🔍 Trying get_predictions():")
        try:
            preds = predictions_obj.get_predictions()
            print(f"get_predictions() type: {type(preds)}")
            if isinstance(preds, dict):
                print(f"Keys: {list(preds.keys())}")
                for key, value in list(preds.items())[:2]:
                    print(f"  {key}: {type(value)}")
                    if isinstance(value, dict):
                        print(f"    Sub-keys: {list(value.keys())}")
            elif isinstance(preds, list):
                print(f"List length: {len(preds)}")
                if preds:
                    print(f"First item: {preds[0]}")
        except Exception as e:
            print(f"get_predictions() error: {e}")

except Exception as e:
    print(f"❌ Error exploring structure: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Enhanced Cross-Validation Prediction Analysis
print("🎯 CROSS-VALIDATION PREDICTION ANALYSIS")
print("=" * 55)

import numpy as np
import re
from collections import defaultdict

def analyze_cv_predictions(predictions_obj):
    """Analyze cross-validation predictions by grouping fold results properly"""

    # Group predictions by base model name and partition type
    model_groups = defaultdict(lambda: defaultdict(list))

    for key, pred_data in predictions_obj._predictions.items():
        model_name = pred_data['model']
        partition = pred_data['partition']

        # Extract base model name (everything before the last underscore + number)
        base_model_match = re.match(r'(.+?)_\d+$', model_name)
        base_model = base_model_match.group(1) if base_model_match else model_name

        # Categorize partition types
        if 'test_fold' in partition:
            partition_type = 'Fold Test'
        elif 'val_fold' in partition:
            partition_type = 'Fold Validation'
        elif 'train_fold' in partition:
            partition_type = 'Fold Train'
        elif 'test' in partition and 'fold' not in partition:
            partition_type = 'Global Test'
        elif 'global_train' in partition:
            partition_type = 'Global Train'
        else:
            partition_type = partition

        model_groups[base_model][partition_type].append({
            'model_instance': model_name,
            'partition': partition,
            'data': pred_data,
            'fold_idx': pred_data.get('fold_idx', 0)
        })

    return model_groups

if 'cv_result' in globals() and hasattr(cv_result, '_predictions'):
    print("📊 Analyzing Cross-Validation Results...")
    print(f"Total prediction records: {len(cv_result._predictions)}")

    # Analyze CV predictions
    cv_analysis = analyze_cv_predictions(cv_result._predictions)

    print(f"Base models found: {list(cv_analysis.keys())}")
    print()

    for base_model, prediction_types in cv_analysis.items():
        print(f"🤖 Base Model: {base_model}")
        print("─" * 50)

        for pred_type, predictions in prediction_types.items():
            print(f"  📈 {pred_type}:")

            if len(predictions) == 1:
                # Single prediction
                pred = predictions[0]
                data = pred['data']
                y_true = np.array(data['y_true']).flatten()
                y_pred = np.array(data['y_pred']).flatten()

                # Calculate metrics
                rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
                r2 = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))
                mae = np.mean(np.abs(y_true - y_pred))

                print(f"    ✅ {len(y_true)} samples")
                print(f"    📊 RMSE: {rmse:.4f}, R²: {r2:.4f}, MAE: {mae:.4f}")
                print(f"    🎯 Model Instance: {pred['model_instance']}")

            else:
                # Multiple fold predictions
                print(f"    ✅ {len(predictions)} fold predictions:")

                all_rmse = []
                all_r2 = []
                all_samples = 0

                for pred in sorted(predictions, key=lambda x: x['fold_idx']):
                    data = pred['data']
                    y_true = np.array(data['y_true']).flatten()
                    y_pred = np.array(data['y_pred']).flatten()

                    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
                    r2 = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

                    all_rmse.append(rmse)
                    all_r2.append(r2)
                    all_samples += len(y_true)

                    fold_info = pred['partition'].split('_')[-1] if '_' in pred['partition'] else pred['fold_idx']
                    print(f"      Fold {fold_info}: {len(y_true)} samples, RMSE={rmse:.4f}, R²={r2:.4f}")
                    print(f"        Instance: {pred['model_instance']}")

                # Show aggregate statistics
                print(f"    📊 Aggregate: {all_samples} total samples")
                print(f"    📈 Average Performance: RMSE={np.mean(all_rmse):.4f}±{np.std(all_rmse):.4f}")
                print(f"                           R²={np.mean(all_r2):.4f}±{np.std(all_r2):.4f}")

            print()

    # Now test combining fold predictions properly
    print("=" * 55)
    print("🧮 TESTING FOLD COMBINATION")
    print("=" * 55)

    # Get base model name for combination
    base_models = list(cv_analysis.keys())
    if base_models:
        test_base_model = base_models[0]
        print(f"Testing fold combination for: {test_base_model}")

        # Find all test fold predictions for this base model
        test_folds = cv_analysis[test_base_model].get('Fold Test', [])

        if test_folds:
            print(f"Found {len(test_folds)} test fold predictions to combine")

            # Combine manually since each fold has different sample indices
            combined_y_true = []
            combined_y_pred = []
            combined_indices = []

            for fold_pred in test_folds:
                data = fold_pred['data']
                combined_y_true.extend(data['y_true'])
                combined_y_pred.extend(data['y_pred'])
                combined_indices.extend(data.get('sample_indices', []))

            combined_y_true = np.array(combined_y_true)
            combined_y_pred = np.array(combined_y_pred)

            # Calculate combined performance
            combined_rmse = np.sqrt(np.mean((combined_y_true - combined_y_pred) ** 2))
            combined_r2 = 1 - (np.sum((combined_y_true - combined_y_pred) ** 2) /
                              np.sum((combined_y_true - np.mean(combined_y_true)) ** 2))

            print(f"✅ Successfully combined fold predictions:")
            print(f"   📊 Total samples: {len(combined_y_true)}")
            print(f"   📈 Combined RMSE: {combined_rmse:.4f}")
            print(f"   📈 Combined R²: {combined_r2:.4f}")

        else:
            print("⚠️ No test fold predictions found for combination")

else:
    print("❌ No cross-validation results available")
    print("   Please run a cross-validation pipeline first to see comprehensive prediction analysis")

In [ ]:
# Enhanced Prediction Visualizer with Filtering
print("🎯 ENHANCED PREDICTION VISUALIZATION WITH FILTERING")
print("=" * 60)

try:
    if 'pred_dataset' in globals() and hasattr(pred_dataset, '_predictions'):
        # Create enhanced visualizer with dataset name override
        dataset_name = getattr(pred_dataset, 'name', 'Sample NIRS Dataset')
        viz_enhanced = PredictionVisualizer(pred_dataset._predictions, dataset_name_override=dataset_name)

        print(f"✅ Enhanced visualizer created with dataset: {dataset_name}")
        print(f"📊 Total prediction records: {len(viz_enhanced.data)}")

        # Test different filter types
        filter_types = ['all', 'best_only', 'folds_only', 'averaged_only', 'global_only']

        for filter_type in filter_types:
            print(f"\n🔍 Testing filter: {filter_type}")

            # Get filtered data count
            filtered_data = viz_enhanced._filter_data_by_prediction_types(filter_type)
            print(f"   📈 Filtered to {len(filtered_data)} records")

            if len(filtered_data) > 0:
                # Show sample of what's included
                sample_partitions = [pred.get('partition', 'unknown') for pred in filtered_data[:3]]
                print(f"   🎯 Sample partitions: {sample_partitions}")

        print(f"\n✅ All filters tested successfully!")

    else:
        print("❌ No prediction data found. Run the prediction cell first!")

except Exception as e:
    print(f"❌ Error testing enhanced visualizer: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Test Filtering Visualizations
print("🎨 TESTING FILTERED VISUALIZATIONS")
print("=" * 50)

try:
    if 'viz_enhanced' in globals():
        # Test 1: Show all predictions as bar chart
        print("\n📊 1. All Predictions Bar Chart")
        fig1 = viz_enhanced.plot_filtered_predictions(
            prediction_filter='all',
            metric='rmse',
            chart_type='bar',
            figsize=(14, 8)
        )
        plt.show()

        # Test 2: Show only best predictions
        print("\n🏆 2. Best Predictions Only")
        fig2 = viz_enhanced.plot_filtered_predictions(
            prediction_filter='best_only',
            metric='r2',
            chart_type='bar',
            figsize=(12, 6)
        )
        plt.show()

        # Test 3: Show only fold predictions
        print("\n📈 3. Cross-Validation Folds Only")
        fig3 = viz_enhanced.plot_filtered_predictions(
            prediction_filter='folds_only',
            metric='rmse',
            chart_type='bar',
            figsize=(16, 8)
        )
        plt.show()

        # Test 4: Performance matrix with filtering
        print("\n🔥 4. Performance Matrix - Global Predictions Only")
        fig4 = viz_enhanced.plot_performance_matrix(
            metric='r2',
            prediction_filter='global_only',
            figsize=(10, 6),
            show_values=True
        )
        plt.show()

        print("✅ All filtered visualizations completed successfully!")

    else:
        print("❌ Enhanced visualizer not found. Run previous cell first!")

except Exception as e:
    print(f"❌ Error in filtered visualizations: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Comprehensive Prediction Summary with Proper Dataset Names
print("📋 COMPREHENSIVE PREDICTION SUMMARY")
print("=" * 60)

try:
    if 'viz_enhanced' in globals():
        # Show comprehensive summary with fixed dataset names
        summary = viz_enhanced.comprehensive_prediction_summary()
        print(summary)

        # Show how the dataset name override works
        print(f"\n🏷️  Dataset Name Override: {viz_enhanced.dataset_name_override}")
        print(f"📊 Data Records: {len(viz_enhanced.data)}")

        # Demonstrate filtering examples
        print(f"\n🔍 FILTERING EXAMPLES:")
        print("=" * 30)

        filters_to_test = {
            'all': 'All prediction types',
            'best_only': 'Best performing prediction per model',
            'folds_only': 'Cross-validation fold predictions only',
            'averaged_only': 'Averaged predictions only',
            'global_only': 'Global train/test predictions only'
        }

        for filter_name, description in filters_to_test.items():
            filtered_count = len(viz_enhanced._filter_data_by_prediction_types(filter_name))
            print(f"• {filter_name:15} | {description:35} | {filtered_count:2} records")

        print(f"\n✅ Enhanced PredictionVisualizer ready with:")
        print(f"   🎯 Dataset name fixes (no more 'unknown')")
        print(f"   🔍 Filtering capabilities (5 filter types)")
        print(f"   📊 Enhanced visualization methods")
        print(f"   📈 Bar charts for prediction type comparison")

    else:
        print("❌ Enhanced visualizer not found. Run previous cell first!")

except Exception as e:
    print(f"❌ Error in comprehensive summary: {e}")
    import traceback
    traceback.print_exc()